In [1]:
# Dependencies
import tensorflow as tf
import pandas as pd
import numpy as np
import keras
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
# Make results reproducible
#seed = 1235
#np.random.seed(seed)
#tf.set_random_seed(seed)

In [3]:
#Loading the dataset
data_X = np.load('male_array_1.npy')
data_y = np.load('male_array_2.npy')

In [4]:
# 
def formattingData(data_X, data_y, test_size, label):
    data_y = pd.DataFrame(data_y)
    data_y.columns = ['혈당', '수축기혈압', '이완기혈압', '총콜레스테롤','HDL콜레스테롤', 'LDL콜레스테롤', '혈색소', '요단백']

    X = pd.DataFrame(data_X)
    y = data_y[label]

    X[label] = y
    n = len(X[X[label] == 2])
    temp = pd.concat([X[X[label] == 1][:n], X[X[label] == 2][:n]], axis = 0)
    y = pd.get_dummies(temp[label])
    y = np.array(y)
    X = np.array(temp[[0,1,2,3,4,5]])

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size)
    
    n_input = len(X_train[0])
    n_output = len(y_train[0])
    
    return  X_train, X_test, y_train, y_test, n_input, n_output

In [5]:
def InitPlaceholders(n_input, n_output):
    with tf.name_scope("input") as scope:
        input_placeholder = tf.placeholder(shape=[None, n_input], dtype=tf.float32)

    with tf.name_scope("target") as scope:
        target_placeholder = tf.placeholder(shape=[None, n_output], dtype=tf.float32)
        
    keep_prob_placeholder = tf.placeholder("float")
    
    return input_placeholder, target_placeholder, keep_prob_placeholder

In [6]:
# Operations
def Create3LayerDNN(input_placeholder,
                    n_input, n_output,
                    input_layer_nodes,
                    hidden_layer_nodes1, hidden_layer_nodes2, hidden_layer_nodes3,
                    output_layer_nodes):
    
    # first hidden layer
    with tf.name_scope("weight1") as scope:
        w1 = tf.get_variable("w1", shape=[input_layer_nodes, hidden_layer_nodes1], initializer=tf.keras.initializers.he_normal())
    with tf.name_scope("bias1")as scope:
        b1 = tf.Variable(tf.random_normal(shape=[hidden_layer_nodes1]))
    with tf.name_scope("layer1") as scope:
        hidden_output1 = tf.nn.relu(tf.add(tf.matmul(input_placeholder, w1), b1))
        hidden_output1 = tf.nn.dropout(hidden_output1, keep_prob = keep_prob)

    # second hidden layer
    with tf.name_scope("weight2") as scope:
        w2 = tf.get_variable("w2", shape=[hidden_layer_nodes1, hidden_layer_nodes2], initializer=tf.keras.initializers.he_normal())
    with tf.name_scope("bias2") as scope:
        b2 = tf.Variable(tf.random_normal(shape=[hidden_layer_nodes2]))
    with tf.name_scope("layer2") as scope:
        hidden_output2 = tf.nn.relu(tf.add(tf.matmul(hidden_output1, w2), b2))
        hidden_output2 = tf.nn.dropout(hidden_output2, keep_prob = keep_prob)

    # third hidden layer
    with tf.name_scope("weight3") as scope:
        w3 = tf.get_variable("w3", shape=[hidden_layer_nodes2, hidden_layer_nodes3], initializer=tf.keras.initializers.he_normal())
    with tf.name_scope("bias3") as scope:
        b3 = tf.Variable(tf.random_normal(shape=[hidden_layer_nodes3]))
    with tf.name_scope("layer3") as scope: 
        hidden_output3 = tf.nn.relu(tf.add(tf.matmul(hidden_output2, w3), b3))
        hidden_output3 = tf.nn.dropout(hidden_output3, keep_prob = keep_prob)
        

    # output layer
    with tf.name_scope("weight4") as scope:
        w4 = tf.get_variable("w4", shape=[hidden_layer_nodes3, output_layer_nodes], initializer=tf.keras.initializers.he_normal())
    with tf.name_scope("bias4") as scope:
        b4 = tf.Variable(tf.random_normal(shape=[output_layer_nodes]))
    logits = tf.add(tf.matmul(hidden_output3, w4), b4)
    with tf.name_scope("output_layer") as scope:
        final_output = tf.nn.softmax(logits)
        
    return logits, final_output


In [7]:
#Cost
def CostFunction(logits, target_placeholder):
    with tf.name_scope("cost") as scope:
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels = target_placeholder))
        tf.summary.scalar("loss",loss)
    return loss

In [8]:
# Optimizer
def Optimizer(loss, learning_rate):
    
    with tf.name_scope("optimizer"):
        optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)
        
    return optimizer

In [9]:
# Initialize variables
def OpenSession():
    
    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)
    
    return sess

In [10]:
# Training
def Training(epoch, interval, keep_prob,
             input_placeholder, target_placeholder, keep_prob_placeholder,
             X_train, y_train,
             sess, logdir):
    

    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter(logdir, sess.graph)
    
    print('Training the model...')
    for i in range(1, (epoch + 1)):
        _, summary = sess.run([optimizer, merged_summary], feed_dict = {input_placeholder : X_train, target_placeholder : y_train, keep_prob_placeholder: keep_prob})
        writer.add_summary(summary,i)
        if i % interval == 0:
            print('Epoch', i, '|', 'Loss:', sess.run(loss, feed_dict={input_placeholder : X_train, target_placeholder : y_train, keep_prob_placeholder: keep_prob}))


In [11]:
def Prediction(mysize, sess, final_output,
              input_placeholder, keep_prob_placeholder):
    print('Predicted:', (sess.run(final_output, feed_dict={input_placeholder: [mysize], keep_prob_placeholder : 1.0})))

In [12]:
def Evaluation(X_test, y_test, sess, final_output,
              input_placeholder, keep_prob_placeholder):
    count = 0
    pred = []
    for i in range(len(X_test)):
        if np.argmax(y_test[i]) == np.argmax(sess.run(final_output, feed_dict={input_placeholder : [X_test[i]], keep_prob_placeholder:1.0})):
            count += 1
        pred.append(np.argmax(sess.run(final_output, feed_dict={input_placeholder : [X_test[i]], keep_prob_placeholder:1.0})))
#    print('accuracy : ', float(count)/len(X_test))
    pred = pd.get_dummies(pred)
    pred = np.array(pred)
    print(classification_report(y_test, pred))

In [13]:
#config

test_size = 0.25
input_layer_nodes = 6
hidden_layer_nodes1 = 30
hidden_layer_nodes2 = 30
hidden_layer_nodes3 = 30
output_layer_nodes =  2

learning_rate = 0.01
keep_prob = 0.5
epoch = 2000
interval = 100

In [14]:
# 혈당 데이터 분석

label = '혈색소'

X_train, X_test, y_train, y_test, n_input, n_output = formattingData(data_X, data_y, test_size, label)

input_placeholder, target_placeholder, keep_prob_placeholder = InitPlaceholders(n_input, n_output)

logits, final_output = Create3LayerDNN(input_placeholder,
                                       n_input, n_output,
                                       input_layer_nodes,
                                       hidden_layer_nodes1, hidden_layer_nodes2, hidden_layer_nodes3,
                                       output_layer_nodes)

loss = CostFunction(logits, target_placeholder)

optimizer = Optimizer(loss, learning_rate)

sess = OpenSession()

Training(epoch, interval, keep_prob,
             input_placeholder, target_placeholder, keep_prob_placeholder,
             X_train, y_train,
             sess, logdir = "tensorboard/"+str(label))

Evaluation(X_test, y_test, sess, final_output,
          input_placeholder, keep_prob_placeholder)

Training the model...
('Epoch', 100, '|', 'Loss:', 0.6542043)
('Epoch', 200, '|', 'Loss:', 0.6280074)
('Epoch', 300, '|', 'Loss:', 0.60743517)
('Epoch', 400, '|', 'Loss:', 0.5922809)
('Epoch', 500, '|', 'Loss:', 0.58449703)
('Epoch', 600, '|', 'Loss:', 0.57491386)
('Epoch', 700, '|', 'Loss:', 0.5708803)
('Epoch', 800, '|', 'Loss:', 0.5660066)
('Epoch', 900, '|', 'Loss:', 0.56511176)
('Epoch', 1000, '|', 'Loss:', 0.5679041)
('Epoch', 1100, '|', 'Loss:', 0.56427133)
('Epoch', 1200, '|', 'Loss:', 0.5611055)
('Epoch', 1300, '|', 'Loss:', 0.56290555)
('Epoch', 1400, '|', 'Loss:', 0.55988824)
('Epoch', 1500, '|', 'Loss:', 0.5626436)
('Epoch', 1600, '|', 'Loss:', 0.55886036)
('Epoch', 1700, '|', 'Loss:', 0.55997735)
('Epoch', 1800, '|', 'Loss:', 0.55922145)
('Epoch', 1900, '|', 'Loss:', 0.55962634)
('Epoch', 2000, '|', 'Loss:', 0.5584874)
             precision    recall  f1-score   support

          0       0.71      0.74      0.73      5169
          1       0.73      0.70      0.71      5